In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [96]:
def compute_entropy(y):
    if len(y) == 0:
        return 0
    probPos = np.sum(y)/len(y)
    if probPos == 0 or probPos == 1:
        return 0
    return -probPos*np.log2(probPos) - (1 - probPos)*np.log2((1 - probPos))

In [31]:
def split_feature(X, node_indices, feature):
    node_features = X[node_indices]
    node_indices = np.asarray(node_indices)
    left_split = node_indices[np.where(node_features[:, feature] == 0)[0]]
    right_split = node_indices[np.where(node_features[:, feature] == 1)[0]]
    return left_split, right_split

In [81]:
def compute_information_gain(X, y, left_indices, right_indices):
    node_indices = np.concatenate((left_indices, right_indices))
    node_features, node_classes = X[node_indices], y[node_indices]
    left_classes, right_classes = y[left_indices], y[right_indices]
    base_entropy = compute_entropy(y[node_indices])
    left_entropy, right_entropy = compute_entropy(left_classes), compute_entropy(right_classes)

    weighted_entropy = (len(left_classes)/len(node_classes) * left_entropy) + (len(right_classes)/len(node_classes) * right_entropy)
    return base_entropy - weighted_entropy


In [76]:
class DecisionNode:
    def __init__(self, value=None, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def getLeft(self):
        return self.left

    def getRight(self):
        return self.right

In [145]:
class DecisionTree:
    def __init__(self, max_depth=4):
        self.root = None
        self.nodes = [self.root]
        self.max_depth = max_depth
        self.current_depth = 1

    def fit_helper(self, x, y, node_indices, minimum_info_gain= 0.01):
        if self.current_depth > self.max_depth:
            positive = np.sum(y[node_indices] == 1)
            negative = len(y[node_indices] == 0) - positive
            return 1 if positive > negative else 0

        best_split = {"feature": -1, "information_gain": -1, "split_tuple": (None, None)}
        for i in range(x.shape[1]):
            left_indices, right_indices = split_feature(x, node_indices, i)
            info_gain = compute_information_gain(x, y, left_indices, right_indices)
            if info_gain > best_split["information_gain"]:
                best_split = {"feature": i, "information_gain": info_gain, "split_tuple": (left_indices, right_indices)}

        if best_split["information_gain"] <= minimum_info_gain:
            positive = np.sum(y[node_indices] == 1)
            negative = len(y[node_indices] == 0) - positive
            return 1 if positive > negative else 0

        node = DecisionNode(value=best_split["feature"])
        node.left = self.fit_helper(x, y, best_split["split_tuple"][0], minimum_info_gain=minimum_info_gain)
        node.right = self.fit_helper(x, y, best_split["split_tuple"][1], minimum_info_gain=minimum_info_gain)
        return node
    
    def fit(self, x, y):
        self.root = self.fit_helper(x, y, range(x.shape[0]))

    def predict_recursive(self, x, node):
        if type(node) == int or not node:
            return node

        feature_considered = node.value
        if x[feature_considered] == 1:
            return self.predict_recursive(x, node.right)
        else:
            return self.predict_recursive(x, node.left)

    def predict(self, x):
        y = np.apply_along_axis(lambda mu: self.predict_recursive(mu, self.root), axis=1, arr=x)
        return y




In [146]:
model = DecisionTree()
model.fit(X_train, y_train)

In [147]:
inputNew = np.asarray([[0, 1, 0], [1, 1, 0]])
print(model.predict(inputNew))

[1 1]


In [56]:
left_indices_test, right_indices_test = split_feature(X_train, range(10), 2)
compute_information_gain(X_train, y_train, left_indices_test, right_indices_test)

1.0


0.2780719051126377